In [27]:
import pandas as pd

# Carregar a planilha
df = pd.read_excel("D:\\Projetos\\AlzheimerEarlyDetection_v3\\datasets\\oasis_cross-sectional.xlsx")

# Exibir as primeiras linhas
print(df.head())


              ID M/F Hand  Age  Educ  SES  MMSE  CDR  eTIV   nWBV    ASF  \
0  OAS1_0001_MR1   F    R   74   2.0  3.0  29.0  0.0  1344  0.743  1.306   
1  OAS1_0002_MR1   F    R   55   4.0  1.0  29.0  0.0  1147  0.810  1.531   
2  OAS1_0003_MR1   F    R   73   4.0  3.0  27.0  0.5  1454  0.708  1.207   
3  OAS1_0004_MR1   M    R   28   NaN  NaN   NaN  NaN  1588  0.803  1.105   
4  OAS1_0005_MR1   M    R   18   NaN  NaN   NaN  NaN  1737  0.848  1.010   

   Delay  
0    NaN  
1    NaN  
2    NaN  
3    NaN  
4    NaN  


In [28]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
num_cols = ["Age", "Educ", "SES", "MMSE", "CDR", "eTIV", "nWBV", "ASF"]
df[num_cols] = scaler.fit_transform(df[num_cols])


In [29]:
import os
import torch
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

# Definir transformações para as imagens
transform = transforms.Compose([
    transforms.Resize((232, 232)),  # Redimensionar para um tamanho fixo
    transforms.ToTensor(),          # Converter para tensor
    transforms.Normalize((0.5,), (0.5,))  # Normalizar
])

# Caminho do diretório com as imagens
dataset_path = "D:\\Projetos\\AlzheimerEarlyDetection_v3\\datasets"

# Criar dataset
dataset = ImageFolder(root=dataset_path, transform=transform)

# Criar DataLoader
batch_size = 32
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


In [30]:
import torch
import torch.nn as nn
import torchvision.models as models

class CustomResNet(nn.Module):
    def __init__(self, num_tabular_features, num_classes):
        super(CustomResNet, self).__init__()
        
        # Carregar ResNet50 pré-treinada
        self.resnet = models.resnet50(weights="IMAGENET1K_V2")
        
        # Remover a última camada totalmente conectada
        num_ftrs = self.resnet.fc.in_features
        self.resnet.fc = nn.Identity()  # Remove a camada final

        # Camada para os dados tabulares
        self.tabular_fc = nn.Sequential(
            nn.Linear(num_tabular_features, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU()
        )

        # Combinar os dois conjuntos de features
        self.final_fc = nn.Linear(num_ftrs + 64, num_classes)

    def forward(self, image, tabular_data):
        img_features = self.resnet(image)  # Extrai features da imagem
        tabular_features = self.tabular_fc(tabular_data)  # Processa os dados tabulares
        combined = torch.cat((img_features, tabular_features), dim=1)  # Concatena as features
        output = self.final_fc(combined)  # Passa pela última camada
        return output
    
    def __getitem__(self, index):
        image = self.images[index]
        tabular_data = self.tabular_data[index]
        label = self.labels[index]
        return image, tabular_data, label


In [31]:
from torchvision import models

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

num_tabular_features = 8  # Ajuste conforme os dados tabulares que você está usando
num_classes = 4  # Número de classes (MildDemented, ModerateDemented, etc.)

model = CustomResNet(num_tabular_features, num_classes).to(device)

In [32]:
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR

criterion = nn.CrossEntropyLoss()
num_epochs = 10
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = StepLR(optimizer, step_size=5, gamma=0.5)  # Reduz LR a cada 5 epochs

for epoch in range(num_epochs):
    model.train()
    
    for images, tabular_data, labels in dataloader:
        images, tabular_data, labels = images.to(device), tabular_data.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images, tabular_data)  # Agora passamos os dois inputs
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}, Loss: {loss.item()}")


ValueError: not enough values to unpack (expected 3, got 2)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Criar matriz de correlação
correlation_matrix = df[num_cols].corr()

# Plotar o heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Correlação entre as variáveis tabulares")
plt.show()
